<a href="https://www.kaggle.com/code/remonboshra/yoga-poses-classification-model?scriptVersionId=97737791" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# **Classification of Yoga posts**

Import Libraries

In [ ]:
import os 
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path
from matplotlib.image import imread
import plotly.express as px 

In [ ]:
tf.config.list_physical_devices()  # APU device is visible to TensorFlow.

In [ ]:
base_dir = '../input/yoga-pose-image-classification-dataset/dataset/'
print('Number of post to be predicted: ',len(os.listdir(base_dir)))

In [ ]:
class_dir = '../input/yoga-pose-image-classification-dataset/dataset/janu sirsasana'


Viualize some of the Data

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=3,figsize=(10,5), subplot_kw={'xticks':[], 'yticks':[]})
for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(os.path.join(class_dir,os.listdir(class_dir)[i])))
    ax.set_title(os.listdir(base_dir)[1])
plt.tight_layout()
plt.show()


In [ ]:
## count number of images in each class
DF = pd.DataFrame(columns=['class','count'])
DF['class']=pd.Series([os.listdir(base_dir)[x] for x in range(0,107)])
DF['count']=pd.Series([len(os.listdir(os.path.join(base_dir,os.listdir(base_dir)[x]))) for x in range(0,107)])


In [ ]:
import seaborn as sns
plt.figure(figsize=(14,10))
g=sns.barplot(x='class', y='count',data=DF)
g.set_xticklabels(g.get_xticklabels(), rotation=90)
plt.tight_layout()

# CNN Model

Data Preprocessing

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import RandomRotation

train_datagen = ImageDataGenerator(rescale=1./255,
                                  width_shift_range = 0.1,
                                  height_shift_range = 0.1,
                                  shear_range = 0.1,
                                  zoom_range= 0.1,
                                  horizontal_flip=True,
                                  fill_mode='nearest',
                                  validation_split=0.15)


In [ ]:
train_data = train_datagen.flow_from_directory(
    base_dir,
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training')

validation_data = train_datagen.flow_from_directory(
    base_dir,
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation')

In [ ]:
from tensorflow.keras.layers import MaxPool2D, Flatten, Dense, Conv2D
# covnet
model = tf.keras.models.Sequential([Conv2D(128,(3,3),input_shape=(224,224,3),activation='relu'),
                                   BatchNormalization(),
                                   Conv2D(128,(3,3)),
                                   BatchNormalization(),
                                   MaxPool2D(2,2),
                                   Conv2D(64,(3,3)),
                                   BatchNormalization(),
                                   Conv2D(64,(3,3)),
                                   BatchNormalization(),
                                   MaxPool2D(2,2),
                                   Conv2D(32,(3,3)),
                                   BatchNormalization(),
                                   Conv2D(32,(3,3)),
                                   BatchNormalization(),
                                   MaxPool2D(2,2),
                                   Flatten(),
                                   Dense(1024,activation='relu'),
                                   BatchNormalization(),
                                   Dense(512,activation='relu'),
                                   Dense(107,activation='softmax')])


In [ ]:
#compile
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.RMSprop(), metrics=['accuracy'])

In [ ]:
with tf.device(tf.DeviceSpec(device_type="GPU", device_index='0')):
    history = model.fit(train_data, steps_per_epoch=120,validation_data=validation_data, epochs=30,verbose=1,
                    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',min_delta=0.0001,patience=15,restore_best_weights=True)])

In [ ]:
model.evaluate(validation_data)

In [ ]:
import pickle
# Saving Model History
with open('./history0.pickle', 'wb') as file:
  pickle.dump(history.history, file)

In [ ]:
# load history
history_file = open('./history0.pickle', 'rb')
loaded_history = pickle.load(history_file)

## Old CNN Model

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
                                  width_shift_range = 0.1,
                                  height_shift_range = 0.1,
                                  shear_range = 0.1,
                                  zoom_range= 0.1,
                                  horizontal_flip=True,
                                  fill_mode='nearest',
                                  validation_split=0.2)

In [ ]:
train_data = train_datagen.flow_from_directory(
    base_dir,
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training')

validation_data = train_datagen.flow_from_directory(
    base_dir,
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation')

In [ ]:
from tensorflow.keras.layers import MaxPool2D, Flatten, Dense, Conv2D
# covnet
model = tf.keras.models.Sequential([Conv2D(128,(3,3),input_shape=(224,224,3),activation='relu'),
                                   Conv2D(128,(3,3)),
                                   MaxPool2D(2,2),
                                   Conv2D(64,(3,3)),
                                   Conv2D(64,(3,3)),
                                   MaxPool2D(2,2),
                                   Conv2D(32,(3,3)),
                                   Conv2D(32,(3,3)),
                                   MaxPool2D(2,2),
                                   Flatten(),
                                   Dense(1024,activation='relu'),
                                   Dense(512,activation='relu'),

In [ ]:
#compile
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])

In [ ]:
with tf.device(tf.DeviceSpec(device_type="GPU", device_index='0')):
    history = model.fit(train_data, steps_per_epoch=120,validation_data=validation_data, validation_steps=35, epochs=25,verbose=1,
                    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',min_delta=0.01,patience=3,restore_best_weights=True)])

In [ ]:
model.evaluate(validation_data)